# Train BlazePalm Model

In [1]:
import os
import glob
import json

import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt

from utils import anchors_generator, encoder, loss_function
from nets import blaze_palm 
from utils.data_generator import DataGenerator

## Create BlazePalm Model

In [2]:
model = blaze_palm.build_blaze_palm_model()

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 128, 128, 32) 320         activation[0][0]                 
______________________________________________________________________________________________

In [4]:
adam = tf.keras.optimizers.Adam()
ssd_loss = loss_function.SSDLoss()
model.compile(optimizer='adam', loss=ssd_loss.compute_loss)

## Prepare Data

In [6]:
train_data_generator = DataGenerator(image_dir=os.path.join('dataset', 'image'), batch_size=32,
                                     annotation_dir=os.path.join('dataset', 'annotation'))
val_data_generator = DataGenerator(image_dir=os.path.join('dataset', 'image'), batch_size=32,
                                     annotation_dir=os.path.join('dataset', 'annotation'))

In [7]:
batch = train_data_generator.__getitem__(0)
ssd_loss.compute_loss(batch[1], model.predict(batch[0]))

<tf.Tensor: id=4283, shape=(32,), dtype=float32, numpy=
array([ 7.3885994, 14.084844 ,  9.368907 , 19.485437 , 12.891543 ,
       10.697935 ,  8.737885 , 17.095488 , 11.3609295,  8.470282 ,
       15.1131115, 16.596268 ,  2.655754 ,  6.5152516, 18.447832 ,
        3.2293603, 14.766701 , 19.689173 , 12.702118 ,  6.7429585,
        9.591715 ,  9.0173   , 12.716931 ,  2.00887  ,  6.5322328,
        6.718907 , 12.158531 , 17.7377   , 13.796915 ,  6.5025263,
        7.323616 , 12.416521 ], dtype=float32)>

## Training

> `fit_generator` not work

In [ ]:
history = model.fit(x=train_data_generator, epochs=100)

Train for 4 steps
Epoch 1/100
4/4 [==============================] - 36s 9s/step - loss: 9.8467
Epoch 2/100
4/4 [==============================] - 31s 8s/step - loss: 9.6170
Epoch 3/100
4/4 [==============================] - 31s 8s/step - loss: 9.4911
Epoch 4/100
3/4 [=====================>........] - ETA: 8s - loss: 9.5480 